In [1]:
import pandas as pd
import urllib.parse
from random import shuffle
from my_rymscraper.rymscraper import rymscraper, RymUrl
network = rymscraper.RymNetwork()

In [2]:
mt = pd.read_csv("meta.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'metacritic_data.csv'

In [ ]:
# mt['tag'] = mt["Singer"] + " - " + mt["Album_Name"];
lower_alnum = lambda s: "".join(x for x in s.lower() if x.isalnum())
mt["id"] = mt["Singer"].apply(lower_alnum) + mt["Release_Year"].apply(str) + mt["Album_Name"].apply(lambda s: lower_alnum(s)[:3] + lower_alnum(s)[-3:])
tags = list(zip(mt["id"], mt["Singer"], mt["Album_Name"]))
# tags = ["Common - A Beautiful Revolution, Vol. 2"]
cols = ['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme', 'Languages', 'Ranked']
df = pd.DataFrame(columns=cols)

In [ ]:
lower_alnum_dash = lambda s: "".join(x for x in s.replace(" ", "-").lower() if (x.isalnum() or x == "-" ))
# tags = [('divmo','Maya Hawke', 'MOSS')]

max_iter = 1 # one iteration is enough for most cases, raise it if errors occur
iter = 0
while iter < max_iter:
  new_tags = []
  for uid, singer, album in tags:
    try:
      # Try url first
      url = "https://rateyourmusic.com/release/album/%s/%s/" % (lower_alnum_dash(singer), lower_alnum_dash(album))
      tag = singer + " - " + album
      album_infos = network.get_album_infos(url=url)
    except:
      album_infos = []
    
    if not album_infos:
      try:
        # Try search then
        url_tag = urllib.parse.quote(tag)
        album_infos = network.get_album_infos(name=url_tag) # get album by search
      except IndexError as e:
        print("❌ ", tag)
      
    if not album_infos:
      new_tags.append((uid, singer, album))
    else:
      print("✅ ", tag)
      album_infos["Id"] = uid
      df = pd.concat([df, pd.DataFrame([album_infos])], axis=0)
    # except IndexError:
    # except (IndexError, AttributeError):

  tags = new_tags
  iter += 1

In [ ]:
xdf = df.reset_index()
xdf = df[cols]
xdf.reset_index()
xdf.describe()

In [ ]:
xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]

/opt/homebrew/anaconda3/envs/edav/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
xdf = xdf[['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme','Ranked']]
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked
count,9,9,9,9,9,9,9,8,9,9,7
unique,9,9,9,9,9,9,7,1,9,9,7
top,beak201833,>>>,Beak>,21 September 2018,3.29\n \n/ 5.00.5\n\n ...,"Experimental Rock, Electronic\nKrautrock, Neo-...",,English,"[The Brazilian, Brean Down, Birthday Suit, Har...","[#c0ab6c, #b06546, #989890, #947e4d, #736945]",#728 for 2018
freq,1,1,1,1,1,1,3,8,1,1,1


In [ ]:
import re
ratings = []
num_ratings = []
for i in xdf["RYM Rating"]:
  if isinstance(i, float):
    ratings.append("")
    num_ratings.append("")
  else:
    ratings.append(float(i.rsplit('\n')[0]))
    num_ = i.rsplit('\n')[-1]
    num = int(re.findall(r"\d*,*\d+",num_)[0].replace(",",""))
    num_ratings.append(num)

In [ ]:
xdf["RYM Rating"] = ratings
xdf["Num_of_RYM_Ratings"] = num_ratings

In [ ]:
xdf

In [ ]:
xdf.to_csv("rym_add.csv")